# Applying BespokeFit to the pom-terphenyl system

This notebook will be used to generate OpenFF force field for the pom-terphenyl system. We use mBuild to construct a short 3-mer of the polymer and get it's charges using AM1-BCC charges using OpenEye. Then using the BespokeFit workflow we will assign torsions/angles. Bespoke fit uses QM calculations on fragments present in our molecule to assign new more accurate torsion potentials.

In [1]:
try:
    from openmm import app
except ImportError:
    from simtk.openmm import app

from openff.toolkit.topology import FrozenMolecule, Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import pdb
import os
import openbabel
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer
import rdkit

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy

In [2]:
comp = mb.load('COc3ccc(c1cccc(C=O)c1)c(c2ccc([C@@H](C)N)cc2)c3', smiles = True, name="POM")
cap_o = mb.load('CO', smiles = True)
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2619 -0.6738 -0.1057]), 1 bonds, id: 140242999318704>
26 <H pos=([-0.3272 -0.5848  0.0369]), 1 bonds, id: 140242999318944>
27 <H pos=([-0.3757 -0.5363 -0.1318]), 1 bonds, id: 140242999319184>
28 <H pos=([-0.3936 -0.3462  0.0261]), 1 bonds, id: 140242999319424>
29 <H pos=([-0.3904 -0.1125  0.0952]), 1 bonds, id: 140242999319664>
30 <H pos=([-0.0188  0.0341  0.2297]), 1 bonds, id: 140242999319904>
31 <H pos=([-0.0211  0.2676  0.3082]), 1 bonds, id: 140242999320144>
32 <H pos=([-0.1868  0.4276  0.2205]), 1 bonds, id: 140242999320384>
33 <H pos=([-0.3797  0.483   0.0736]), 1 bonds, id: 140242999320432>
34 <H pos=([-0.3514  0.1181 -0.0307]), 1 bonds, id: 140242999337312>
35 <H pos=([ 0.1831 -0.1982  0.0834]), 1 bonds, id: 140242999337552>
36 <H pos=([ 0.3893 -0.0704  0.0507]), 1 bonds, id: 140242999337792>
37 <H pos=([ 0.4012  0.2527 -0.1548]), 1 bonds, id: 140242999338032>
38 <H pos=([0.5627 0.2867 0.0274]), 1 bonds, id: 140242999338272>
39 <H pos=([0.5016 0.1413 0.115 ]), 1

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2929 -0.5741 -0.0682]), 4 bonds, id: 140242999383040> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.1818 -0.4854 -0.0746]), 2 bonds, id: 140242999383136> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1854 -0.3509 -0.0349]), 3 bonds, id: 140242999383712> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[33, 42],
                  separation=.15,
                  replace=True,
                  # orientation = [[1,0,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=3, sequence='A')
chain.energy_minimize(forcefield="GAFF")

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2897 -0.5721 -0.068 ]), 4 bonds, id: 140242718863424> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.1829 -0.4886 -0.0717]), 2 bonds, id: 140242718884384> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.189  -0.3486 -0.0396]), 3 bonds, id: 140242718884480> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in ReadMolecule
  This Mol2 file is non-standard. Problem with molecule: RES Cannot interpret atom types correctly, instead attempting to interpret atom type: O as elements instead.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in ReadMolecule
  This Mol2 file is non-standard. Problem with molecule: RES Cannot interpret atom types correctly, instead attempting to interpret atom type: O as elements instead.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in R

In [17]:
# Set residue labels for polymer
for label in chain.labels["monomer"]:
    label.name = "TRI"
for label in chain.labels["Compound"]:
    label.name = "CAP"


In [18]:
# Rename atoms using mbuild interface
counts = {}
for particle in chain.particles():
    atom_name = particle.name
    if not atom_name in counts.keys():
        counts[atom_name] = 1
    else:
        counts[atom_name] += 1
    particle.name = atom_name + str(counts[atom_name])

In [19]:
# Write to PDB
chain.save("pom_trimer_mbuild.pdb", overwrite = True, residues = ["TRI", "CAP"])

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C1 pos=([-0.3119 -0.5416 -0.0534]), 4 bonds, id: 140242718863424> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O1 pos=([-0.1908 -0.4622 -0.0538]), 2 bonds, id: 140242718884384> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C2 pos=([-0.1961 -0.3272 -0.0285]), 3 bonds, id: 140242718884480> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/si

# File conversion between `mbuild` and OpenFF

Ideally structures generated from `mbuild` could be directly imported into the OpenFF workflow, however they are not. We have to recover connectivity information and write structures out into `.mol` and `.pdb` files to put into the OpenFF forcefield assignment workflow.

In [20]:
# Load into RDKit to assign bond infomration in PDB file
rdmol = rdkit.Chem.rdmolfiles.MolFromPDBFile("pom_trimer_mbuild.pdb", removeHs=False)
for atom in rdmol.GetAtoms():
    ri = atom.GetPDBResidueInfo()
    ri.SetIsHeteroAtom(False)
rdkit.Chem.rdmolfiles.MolToPDBFile(rdmol, "pom_trimer_mbuild_bonds.pdb")

In [21]:
# OpenBabel to convert PDB to Mol format
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("pdb", "mol")
mol = openbabel.OBMol()
obConversion.ReadFile(mol, "pom_trimer_mbuild_bonds.pdb")
obConversion.WriteFile(mol, "pom_trimer_mbuild.mol")

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


True

# OpenFF Parameter Assignment

Now that we have our input `.mol` and `.pdb` files we can assign bonded and non-bonded parameters using OpenFF.

In [22]:
# Load in with OpenFF
tri_pom = Molecule.from_file("pom_trimer_mbuild.mol")

In [23]:
pdbfile = app.PDBFile("pom_trimer_mbuild_bonds.pdb")
omm_topology = pdbfile.topology
omm_topology

<Topology; 1 chains, 5 residues, 153 atoms, 161 bonds>

In [24]:
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[tri_pom]
)

In [25]:
# Modified OpenFF to increase maxAtoms for AM1BCC method to 500 atoms
if not os.path.exists('terphenyl_pom_trimer_charges.sdf'):
    tri_pom.assign_partial_charges(partial_charge_method="am1bcc")
    tri_pom.to_file('terphenyl_pom_trimer_charges.sdf', file_format = 'sdf')
else:
    tri_pom = Molecule.from_file('terphenyl_pom_trimer_charges.sdf')

# Submit to BespokeFit Executor

The BespokeFit workflow needs an SDF file with charges to find fragments and assign a bespoke set of force-field parameters for chemical systems. Make sure you have an instance of the Bespoke fit executor running in another terminal by running:

```
BEFLOW_OPTIMIZER_KEEP_FILES=True openff-bespoke executor launch \
    --n-fragmenter-workers 1                                    \
    --n-optimizer-workers  1                                    \
    --n-qc-compute-workers 1                                    \
    --qc-compute-n-cores   1                                    \
    --qc-compute-max-mem   1.5                                  \
    --directory            bespoke-executor
```

These flags determine how BespokeFit distributes processes to perform the bespokefit. The `BEFLOW_OPTIMIZER_KEEP_FILES=True` allows us to look at output files from the fit.

In [26]:
!BEFLOW_OPTIMIZER_KEEP_FILES=True openff-bespoke executor submit \
     --file                 "terphenyl_pom_trimer_charges.sdf" \
     --workflow             "default"                            \
     --default-qc-spec      xtb gfn2xtb none


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

1. preparing the bespoke workflow                                               
                                                                                
⠋ loading the molecules
[✓] 1 molecules found
building fitting schemas ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
[✓] fitting schemas generated
                                                                                
2. submitting the workflow                                                      
                                                                                
submitting tasks ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
[✓] the following workflows were submitted
┏━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ SMILES            ┃ NAME             ┃ FILE                             ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩


In [27]:
!openff-bespoke executor list


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

The following optimizations were found:
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ ID ┃ SMILES                                                        ┃ STATUS  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ 1  │ C[C@H](c1ccc(cc1)c2cc(ccc2c3cccc(c3)C(=O)N[C@H](C)c4ccc(cc4)c │ waiting │
│    │ 5cc(ccc5c6cccc(c6)C(=O)OC)OC)OC)NC(=O)c7cccc(c7)c8ccc(cc8c9cc │         │
│    │ c(cc9)[C@@H](C)NC(=O)OC(C)(C)C)OC                             │         │
└────┴───────────────────────────────────────────────────────────────┴─────────┘


In [28]:
!openff-bespoke executor watch --id 1


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

⠙ fragmenting the moleculele
[✓] fragmentation successful
⠙ generating bespoke QC datata
[✓] qc-generation successful
⠏ optimizing the parametersrs
[✓] optimization successful


In [29]:
!openff-bespoke executor retrieve --id 1 --output "pom_trimer_bespoke_fit.json" --force-field "openff-2.0.0_bespoke_pom_trimer.offxml"


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

[✓] the bespoke fit is finished
                                                                                
outputs have been saved to pom_trimer_bespoke_fit.json                          
                                                                                
                                                                                
the bespoke force field has been saved to openff-2.0.0_bespoke_pom_trimer.offxml
                                                                                
